<a href="https://colab.research.google.com/github/upriyam-cmu/EDGE-Rec/blob/main/execute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install project code

In [ ]:
!pip uninstall -y edge-rec
!pip install -e .

# Train model

In [ ]:
from edge_rec.gaussian_diffusion import GaussianDiffusion, Trainer
from edge_rec.model import GraphReconstructionModel

model = GraphReconstructionModel.default()

diffusion_model = GaussianDiffusion(model, image_size=50)
trainer = Trainer(
    diffusion_model, 
    folder="./movie_lens",
    train_batch_size=16,
    gradient_accumulate_every=1,
    train_lr=1e-4,
    train_num_steps=int(1e4),
    ema_update_every=10,
    ema_decay=0.995,
    adam_betas=(0.9, 0.99),
    save_and_sample_every=200,
    full_sample=True,
    num_samples=25,
    n_subsamples=1600,
    results_folder="./results",
    amp=False,
    mixed_precision_type='fp16',
    split_batches=True,
    max_grad_norm=1.,
    save_best_and_latest_only=False,
    train_on_full_graph=False,
    use_alternate_dense_dataset=True,
    train_on_binary_targets=False,
    train_mask_unknown_ratings=False,
    ml_100k=False,
)
print("Using device:", trainer.device)

In [ ]:
trainer.train()

# Sample ratings

In [ ]:
# d70_subgraph, train_ratings, test_ratings = trainer.ds.dataset.get_subgraph(
#     subgraph_size=50,
#     target_density=0.7,
#     include_separate_train_test_ratings=True
# )
full_graph, train_ratings, test_ratings = trainer.ds.build_feat_graph(include_separate_train_test_ratings=True)
sampled_ratings = trainer.eval(full_graph=full_graph[:-1], milestone=2200, subgraph_size=50)[0, 0, :, :].cpu().detach()
sampled_ratings = sampled_ratings * 2.5 + 3

# Evaluate metrics

In [ ]:
from edge_rec.evaluate import get_metrics
import torch

train_edges = train_ratings.long().to_sparse_coo()
train_edges = torch.cat([
    train_edges.indices(),
    train_edges.values().unsqueeze(dim=0)
], dim=0)
test_edges = test_ratings.long().to_sparse_coo()
test_edges = torch.cat([
    test_edges.indices(),
    test_edges.values().unsqueeze(dim=0)
], dim=0)

test_metrics = get_metrics(train_edges, test_edges, sampled_ratings)
test_metrics

# Display sampled ratings distribution

In [ ]:
import matplotlib.pyplot as plt

plt.hist(sampled_ratings.numpy().flatten(), bins=5)
plt.show()